In [1]:
import trimesh
import kaolin
from csdf import index_vertices_by_faces
import torch
import os
import csdf

In [2]:
os.environ["CUDA_VISIBLE_DIVICES"] = "1"

In [3]:
device = "cuda"

In [4]:
sphere_mesh = trimesh.primitives.Sphere(radius=1)
samples, _ = trimesh.sample.sample_surface(sphere_mesh, 10000)
verts = torch.Tensor(sphere_mesh.vertices).to(device).unsqueeze(0)
faces = torch.Tensor(sphere_mesh.faces).long().to(device)
face_verts = index_vertices_by_faces(verts, faces)


/tmp/ipykernel_712835/2183168084.py:3: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811803361/work/torch/csrc/utils/tensor_numpy.cpp:189.)
  verts = torch.Tensor(sphere_mesh.vertices).to(device).unsqueeze(0)


In [5]:
for i in range(50):
    x = torch.Tensor(samples).to(device).requires_grad_()
    x = x*1.01
    signs = kaolin.ops.mesh.check_sign(verts, faces, x.unsqueeze(0))
    dis, dis_faces, dis_types = kaolin.metrics.trianglemesh.point_to_mesh_distance(
        x.unsqueeze(0), face_verts)
    dis = torch.sqrt(dis + 1e-8)
    dis = torch.where(signs, -dis, dis)
    g = torch.autograd.grad([dis.sum()], [x], create_graph=True,
                            retain_graph=True)[0]


In [6]:
for i in range(50):
    x = torch.Tensor(samples).to(device).requires_grad_()
    x = x*1.01
    signs = kaolin.ops.mesh.check_sign(verts, faces, x.unsqueeze(0))
    dis_, dis_faces, dis_types = csdf.compute_sdf(
        x, face_verts.squeeze(0))
    dis_ = torch.sqrt(dis_ + 1e-8)
    dis_ = torch.where(signs, -dis_, dis_)
    g_ = torch.autograd.grad([dis_.sum()], [x], create_graph=True,
                            retain_graph=True)[0]
